# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('../Рекомендательные системы/retail_train.csv')
item_features = pd.read_csv('../Рекомендательные системы/product.csv')
user_features = pd.read_csv('../Рекомендательные системы/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
n_items_before = data_train_lvl_1['item_id'].nunique()

In [4]:
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

In [5]:
n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [6]:
recommender = MainRecommender(data_train_lvl_1)

In [8]:
recommender.get_als_recommendations(2375, N=200)

[899624,
 1106523,
 1044078,
 883932,
 1116376,
 925862,
 871756,
 1046545,
 5569230,
 850925,
 963727,
 1025535,
 1092937,
 9836106,
 844179,
 12262978,
 879504,
 1096573,
 832678,
 896862,
 1004390,
 965766,
 841220,
 870547,
 1000753,
 865528,
 12301839,
 1119454,
 832442,
 8090537,
 999714,
 947858,
 865456,
 902172,
 12731544,
 888543,
 1009770,
 1051323,
 1059902,
 1000736,
 986912,
 944534,
 923149,
 854852,
 5568721,
 936355,
 1081177,
 823704,
 822178,
 1024319,
 1031316,
 835300,
 1003616,
 5569845,
 1072685,
 1026984,
 1134678,
 910032,
 1029743,
 8090521,
 9835223,
 1119784,
 830304,
 1004906,
 973181,
 1001702,
 6534480,
 999779,
 834911,
 12731517,
 827919,
 891423,
 1056509,
 1022827,
 1133312,
 1092885,
 896085,
 896613,
 1006878,
 861279,
 9526410,
 851188,
 952163,
 1100972,
 1104349,
 845208,
 925178,
 938138,
 823990,
 1090931,
 13841744,
 949965,
 998556,
 863885,
 1016800,
 847790,
 1038663,
 1068719,
 8090539,
 7441873,
 1097458,
 15926886,
 999270,
 1103691,
 10

In [9]:
recommender.get_own_recommendations(2375, N=200)

[948640,
 918046,
 847962,
 907099,
 873980,
 884694,
 10285454,
 1107760,
 7169090,
 979674,
 10308345,
 1069531,
 974766,
 1015474,
 950935,
 847066,
 1102207,
 1020770,
 9521787,
 974265,
 940996,
 8019845,
 5567194,
 12811490,
 1003616,
 973181,
 890719,
 982955,
 9677152,
 998519,
 1072685,
 1131382,
 1021715,
 12263119,
 960791,
 7441873,
 986021,
 956666,
 1038692,
 9677748,
 9297223,
 927030,
 12757653,
 1046919,
 6391532,
 989069,
 1068451,
 951954,
 835300,
 937343,
 1047249,
 13876348,
 1061732,
 981601,
 1121028,
 1087547,
 828393,
 996269,
 951951,
 1036093,
 1023815,
 5570408,
 827667,
 1082454,
 1006878,
 5570048,
 841309,
 1078652,
 1115553,
 1056492,
 1138467,
 1004945,
 947858,
 1092885,
 1121694,
 938138,
 8019916,
 827919,
 984315,
 10341855,
 883932,
 8291322,
 1096794,
 1028938,
 1087618,
 8020166,
 1082185,
 866871,
 930666,
 825994,
 910151,
 823990,
 848029,
 896613,
 12301839,
 1117219,
 1135258,
 869868,
 1046545,
 899624,
 6442594,
 1137775,
 825343,
 104290

In [10]:
recommender.get_similar_items_recommendation(2375, N=200)

[1046545,
 1044078,
 823990,
 1115576,
 15778319,
 1122358,
 9677620,
 947858,
 885863,
 1025535,
 863447,
 878996,
 906923,
 871611,
 1119454,
 933637,
 828106,
 1018740,
 1127831,
 1098892,
 938138,
 1138467,
 858373,
 12301100,
 899624,
 6533581,
 1028747,
 5564303,
 985999,
 957411,
 9835903,
 1127025,
 1000753,
 9836526,
 1075283,
 993912,
 918994,
 1055503,
 15926886,
 9445820,
 10456152,
 916122,
 983584,
 929768,
 9836752,
 1035676,
 1110244,
 904105,
 8065410,
 1008032,
 954651,
 1115098,
 13008459,
 12301839,
 822225,
 1002771,
 9245106,
 828106,
 1016276,
 1105488,
 1121360,
 825343,
 898068,
 1038998,
 967994,
 980263,
 8090537,
 882247,
 5568378,
 948650,
 1046488,
 12262778,
 845677,
 846986,
 1081262,
 969945,
 968164,
 890909,
 1133312,
 936470,
 999270,
 12301109,
 1055503,
 12132277,
 1092730,
 858999,
 5592557,
 1054262,
 13008334,
 1103618,
 951227,
 12757425,
 958382,
 1084613,
 870315,
 1127025,
 1072519,
 937292,
 885863,
 1087755,
 868909,
 1087102,
 1110857,
 1

In [11]:
recommender.get_similar_users_recommendation(2375, N=200)

[1124971,
 10457044,
 820612,
 1129805,
 1012801,
 1068865,
 1102003,
 921406,
 935578,
 5574377,
 12523928,
 1026945,
 983665,
 882826,
 902640,
 6391134,
 918638,
 5568758,
 917033,
 12523928,
 12352054,
 5707857,
 10198378,
 12427353,
 852015,
 12263857,
 1010308,
 1065259,
 8020166,
 1101502,
 959455,
 965956,
 920025,
 820582,
 920025,
 825317,
 1118946,
 1107760,
 1031316,
 837495,
 948640,
 841365,
 982469,
 1096573,
 6979437,
 1116050,
 958023,
 6391557,
 9677454,
 896757,
 7167903,
 916990,
 875392,
 857130,
 13213202,
 873715,
 1117602,
 1037135,
 920025,
 1134296,
 894360,
 1012801,
 9553048,
 1055403,
 979674,
 1081533,
 969866,
 871514,
 1010051,
 857538,
 1131625,
 1123045,
 956125,
 918046,
 988697,
 10341855,
 836085,
 7409622,
 950894,
 853124,
 887375,
 873324,
 979674,
 9392700,
 1112405,
 1030577,
 1071196,
 1120190,
 974265,
 1102829,
 879699,
 820486,
 911974,
 9837092,
 868645,
 915905,
 949257,
 1057168,
 8019902,
 8019681,
 913249,
 945909,
 1028938,
 6514287,


### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 200 кандидатов (k=200)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [12]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [13]:
cold_users = np.setdiff1d(result_lvl_1['user_id'], data_train_lvl_1['user_id']).tolist()
result_lvl_1 = result_lvl_1[~result_lvl_1['user_id'].isin(cold_users)]

In [17]:
#implicit
result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=200))
result_lvl_1.apply(lambda row: recall_at_k(row['als'], row['actual']), axis=1).mean()

0.09733315079456357

In [18]:
result_lvl_1['own_recommendations'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))
result_lvl_1.apply(lambda row: recall_at_k(row['own_recommendations'], row['actual']), axis=1).mean()

0.13537278412833254

In [19]:
result_lvl_1['similar_items_recommendation'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=200))
result_lvl_1.apply(lambda row: recall_at_k(row['similar_items_recommendation'], row['actual']), axis=1).mean()

0.08474426780395074

Товары, похожие на топ-N купленных юзером товаров показали лучший результат

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [20]:
def new_user_features(data, user_features):
    
    new_user_features = user_features.merge(data, on='user_id', how='left')

    #Чек
    basket = new_user_features.groupby(['user_id'])['sales_value'].sum().reset_index()
    baskets_qnt = new_user_features.groupby('user_id')['basket_id'].count().reset_index()
    baskets_qnt.rename(columns={'basket_id': 'baskets_qnt'}, inplace=True)
    #Средний недельный чек
    average_basket = basket.merge(baskets_qnt)
    average_basket['average_basket'] = average_basket.sales_value / average_basket.baskets_qnt
    average_basket['sum_per_week'] = average_basket.sales_value / new_user_features.week_no.nunique()
    average_basket = average_basket.drop(['sales_value', 'baskets_qnt'], axis=1)
    user_features = user_features.merge(average_basket)
    
    

    return user_features

In [21]:
def new_item_features(data, item_features):
    
    new_item_features = item_features.merge(data, on='item_id', how='left')
    
    # Цена
    price = new_item_features.groupby('item_id')['sales_value'].sum() / new_item_features.groupby('item_id')['quantity'].sum()
    price = price.groupby('item_id').mean().reset_index()
    price.columns = ['item_id', 'price']
    price['price'].fillna(0, inplace= True)
    
    #Количество продаж и среднее количество продаж товара

    item_qnt = new_item_features.groupby(['item_id'])['quantity'].count().reset_index()
    item_qnt.rename(columns={'quantity': 'quantity_of_sales'}, inplace=True)
    item_qnt['quantity_of_sales_per_week'] = item_qnt['quantity_of_sales'] / new_item_features['week_no'].nunique()
    item_features = item_features.merge(item_qnt, on='item_id')
    
    return item_features

In [22]:
item_features = new_item_features(data_train_lvl_2, item_features)

In [23]:
user_features = new_user_features(data_train_lvl_2, user_features)

In [27]:
def train_test_preprocessing(data):    
    users_lvl_2 = pd.DataFrame(data['user_id'].unique())
    users_lvl_2.columns = ['user_id']

    train_users = data_train_lvl_1['user_id'].unique()
    train_users.shape

    users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]
    users_lvl_2_ = users_lvl_2.copy()
    users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
    
    s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'item_id'

    users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
    
    users_lvl_2['flag'] = 1

    targets_lvl_2 = data[['user_id', 'item_id']].copy()
    targets_lvl_2.head(2)

    targets_lvl_2['target'] = 1  

    targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

    targets_lvl_2['target'].fillna(0, inplace= True)
    targets_lvl_2.drop('flag', axis=1, inplace=True)

    targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

    X = targets_lvl_2.drop('target', axis=1)
    y = targets_lvl_2[['target']]
    
    return X, y


In [28]:
X_train, y_train = train_test_preprocessing(data_train_lvl_2)

In [31]:
cat_feats = X_train.columns[2:].tolist()

In [32]:
X_train[cat_feats] = X_train[cat_feats].astype('category')
X_test, y_test = train_test_preprocessing(data_val_lvl_2)
X_test[cat_feats] = X_test[cat_feats].astype('category')

In [33]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

test_preds_proba = lgb.predict_proba(X_test)[:, 1]

D:\Prog\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
D:\Prog\Anaconda3\envs\tensorflow\lib\site-packages\lightgbm\basic.py:863: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


In [34]:
test_preds_proba[:10]

array([0.34877816, 0.34877816, 0.02864643, 0.02864643, 0.00134367,
       0.02848676, 0.00138692, 0.01093452, 0.01754717, 0.35690584])

In [35]:
def get_final_recomendation(X_test, test_preds_proba, data_val_lvl_2):
    X_test['predict_proba'] = test_preds_proba

    X_test.sort_values(['user_id', 'predict_proba'], ascending=False, inplace=True)

    result = X_test.groupby('user_id').head(5)

    recs = result.groupby('user_id')['item_id']
    recomendations = []
    for user, preds in recs:
        recomendations.append({'user_id': user, 'recomendations': preds.tolist()})

    recomendations = pd.DataFrame(recomendations)

    result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
    result_lvl_2.columns=['user_id', 'actual']

    result_lvl_2 = result_lvl_2.merge(recomendations)
    
    return result_lvl_2

In [36]:
result_lvl_2 = get_final_recomendation(X_test, test_preds_proba, data_val_lvl_2)

In [37]:
result_lvl_2.apply(lambda row: precision_at_k(row['recomendations'], row['actual']), axis=1).mean()

0.14294117647058824

### Финальный проект

Мы уже прошли всю необходимую теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - money precision@5. Порог для уcпешной сдачи проекта money precision@5 > 20%

Бизнес ограничения в топ-5 товарах:
- Для каждого юзера 5 рекомендаций (иногда модели могут возвращать < 5)
- **2 новых товара** (юзер никогда не покупал)
- **1 дорогой товар, > 7 долларов**
- **Все товары из разных категорий** (категория - department)  
- **Стоимость каждого рекомендованного товара > 1 доллара**  

- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и .csv файл с рекомендациями. В .csv файле 2 столбца: user_id - (item_id1, item_id2, ..., item_id5)